<a href="https://colab.research.google.com/github/terose73/ICLab/blob/main/ICLab_Analysis_ter9hb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analyzing ICLab Censorship Data**

(1) measuring the type of censorship 

(2) measuring the change of censorship over time

(3) visualizing the censorship (e.g., location, scale) 


idea 1. map AS # to names
-> see which AS's censor the most sites

idea 2. measure censorship by website type (business, abortion, finance, etc.)

idea 3. map country codes to full country name

idea 4. see what countries censor (what website type)

**Installation / Google Drive Linking**

In [1]:
import sklearn
import sys

!{sys.executable} -m pip install pycountry
!{sys.executable} -m pip install -U libra
!{sys.executable} -m pip install urllib

import pycountry
import urllib
from libra import client
import numpy as np 
import pandas as pd
import os 
import collections

np.random.seed(2001) 

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

     |████████████████████████████████| 10.1MB 7.6MB/s 
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746865 sha256=42f2efd7fad1afa5932acaf9733c6e1ef19d9788861df7f09cd15e77ee9c5add
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built pycountry
     |████████████████████████████████| 92kB 5.4MB/s 
     |████████████████████████████████| 624kB 10.6MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 102kB 938kB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 7.1MB 9.4MB/s 
     |████████████████████████████████| 320.4MB 51kB/s 
     |████████████████████████████████| 778kB 43.2MB/s 
     |████████████████████████████████| 112kB 43.4MB/s 
     |████████████████████████████████| 133kB 49.2MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |██████████████████████████████

In [2]:
# Run this once you have added the ICLab data (Add shortcut to 'My Drive')
# https://drive.google.com/drive/folders/1jjSNjIIB0EWDachhaheHePXjvLQeNHkr

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# make sure there are csv files in the directory
!ls "/content/drive/My Drive/ICLab Data for S&P20 paper"

 iclab_2017-01.csv   iclab_2017-09.csv	 iclab_2018-05.csv
 iclab_2017-02.csv   iclab_2017-10.csv	 iclab_2018-06.csv
 iclab_2017-03.csv   iclab_2017-11.csv	 iclab_2018-07.csv
 iclab_2017-04.csv   iclab_2017-12.csv	 iclab_2018-08.csv
 iclab_2017-05.csv   iclab_2018-01.csv	 iclab_2018-09.csv
 iclab_2017-06.csv   iclab_2018-02.csv	'URL classification'
 iclab_2017-07.csv   iclab_2018-03.csv
 iclab_2017-08.csv   iclab_2018-04.csv


In [4]:
# Read in the data from My Drive symbolic link

dir = '/content/drive/My Drive/ICLab Data for S&P20 paper'

df = pd.read_csv(dir + '/iclab_2018-09.csv')

In [5]:
# Provide a quick summary of the data (features, count, type, entries)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052887 entries, 0 to 1052886
Data columns (total 17 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   filename          1052887 non-null  object 
 1   server_t          1052887 non-null  object 
 2   country           1052887 non-null  object 
 3   as_number         1034724 non-null  float64
 4   schedule_name     1052887 non-null  object 
 5   url               1052887 non-null  object 
 6   dns               735638 non-null   object 
 7   dns_reason        267952 non-null   object 
 8   dns_all           998828 non-null   object 
 9   dns_reason_all    249893 non-null   object 
 10  http_status       898415 non-null   float64
 11  block             898415 non-null   object 
 12  body_len          898415 non-null   float64
 13  http_reason       898415 non-null   object 
 14  packet_updated    45244 non-null    object 
 15  packet_reason     45244 non-null    object 
 16  

In [6]:
df.describe()

,as_number,http_status,body_len
count,1.034724e+06,898415.000000,8.984150e+05
mean,7.289921e+04,179.898106,8.469376e+04
std,7.523522e+04,111.618692,1.663344e+05
min,1.249000e+03,-1.000000,0.000000e+00
25%,1.298900e+04,200.000000,1.064000e+03
50%,4.299400e+04,200.000000,2.341500e+04
75%,6.300800e+04,200.000000,9.106900e+04
max,3.951110e+05,999.000000,3.628628e+06


This is not entirely useful as *as_number* and *http_status* are discrete, but *body_len* may be used for censorship or show some interesting underlying patterns.

**Data Cleanup and Feature Engineering**

In [7]:
pd.set_option('display.max_colwidth', 0)
df.agg(['unique']).transpose()

unique
filename          [baseline-2018-09-01T000154.371069.json.bz2, baseline-2018-10-02T011527.678225.json.bz2, baseline-2018-09-01T002206.909253.json.bz2, baseline-2018-09-01T000211.982577.json.bz2, baseline-2018-09-01T012805.179303.json.bz2, baseline-2018-09-01T000348.154035.json.bz2, baseline-2018-09-29T122200.922828.json.bz2, baseline-2018-09-01T001914.221847.json.bz2, baseline-2018-09-01T002347.327314.json.bz2, baseline-2018-09-01T022556.188689.json.bz2, baseline-2018-09-01T012507.257001.json.bz2, baseline-2018-09-16T074206.498584.json.bz2, baseline-2018-09-29T015739.798070.json.bz2, baseline-2018-09-01T012755.940830.json.bz2, baseline-2018-09-01T014549.549549.json.bz2, baseline-2018-09-01T031201.944294.json.bz2, baseline-2018-09-01T031910.129586.json.bz2, baseline-2018-09-01T041723.359026.json.bz2, baseline-2018-09-18T112730.485175.json.bz2, baseline-2018-09-01T032809.041273.json.bz2, baseline-2018-09-01T043033.228301.json.bz2, baseline-2018-09-01T033020.601453.json.bz2, baseline-2018-09-29T183142.687279.json.bz2, baseline-2018-09-01T042033.228432.json.bz2, baseline-2018-09-01T055009.914886.json.bz2, baseline-2018-09-01T045208.307428.json.bz2, baseline-2018-09-01T064204.770879.json.bz2, baseline-2018-09-01T062346.951665.json.bz2, baseline-2018-09-25T085727.507107.json.bz2, baseline-2018-09-01T070700.369570.json.bz2, baseline-2018-09-01T080554.927843.json.bz2, baseline-2018-09-01T080609.035147.json.bz2, baseline-2018-09-01T084135.726037.json.bz2, baseline-2018-09-01T094301.914785.json.bz2, baseline-2018-09-01T100717.985693.json.bz2, baseline-2018-09-01T094813.042643.json.bz2, baseline-2018-09-01T084221.770008.json.bz2, baseline-2018-09-01T093527.657129.json.bz2, baseline-2018-09-01T104617.083913.json.bz2, baseline-2018-09-01T115336.125062.json.bz2, baseline-2018-09-01T110413.611776.json.bz2, baseline-2018-09-01T112734.851861.json.bz2, baseline-2018-09-01T122429.615098.json.bz2, baseline-2018-09-01T130937.370936.json.bz2, baseline-2018-09-01T132549.694038.json.bz2, baseline-2018-09-01T142921.828216.json.bz2, baseline-2018-09-01T133024.580958.json.bz2, baseline-2018-09-01T135130.512703.json.bz2, baseline-2018-09-01T153242.171205.json.bz2, baseline-2018-09-01T155959.708269.json.bz2, baseline-2018-09-01T145819.006208.json.bz2, baseline-2018-09-01T161700.324482.json.bz2, baseline-2018-09-01T152133.399919.json.bz2, baseline-2018-09-01T162506.050253.json.bz2, baseline-2018-09-01T164325.603238.json.bz2, baseline-2018-09-01T155840.772808.json.bz2, baseline-2018-09-01T170158.251843.json.bz2, baseline-2018-09-01T155931.769552.json.bz2, baseline-2018-09-24T214632.291619.json.bz2, baseline-2018-09-01T170123.077746.json.bz2, baseline-2018-09-16T084328.850857.json.bz2, baseline-2018-09-01T175721.886001.json.bz2, baseline-2018-09-01T172643.354094.json.bz2, baseline-2018-09-01T183403.564449.json.bz2, baseline-2018-09-20T003743.820612.json.bz2, baseline-2018-09-29T221602.443699.json.bz2, baseline-2018-09-29T210024.601853.json.bz2, baseline-2018-09-01T185540.188463.json.bz2, baseline-2018-09-01T173925.731773.json.bz2, baseline-2018-09-01T200019.659990.json.bz2, baseline-2018-09-01T224830.438845.json.bz2, baseline-2018-09-01T215218.852063.json.bz2, baseline-2018-09-01T223306.519698.json.bz2, baseline-2018-09-01T233135.165997.json.bz2, baseline-2018-09-01T230625.089331.json.bz2, baseline-2018-09-02T001809.925976.json.bz2, baseline-2018-09-02T013758.958945.json.bz2, baseline-2018-09-02T002126.252146.json.bz2, baseline-2018-09-02T002131.177627.json.bz2, baseline-2018-09-02T013818.909647.json.bz2, baseline-2018-09-02T002610.042334.json.bz2, baseline-2018-09-17T144653.568304.json.bz2, baseline-2018-09-02T010613.888775.json.bz2, baseline-2018-09-02T020907.268134.json.bz2, baseline-2018-09-02T024643.458483.json.bz2, baseline-2018-09-21T170739.291503.json.bz2, baseline-2018-09-02T023737.334076.json.bz2, baseline-2018-09-02T032356.148063.json.bz2, baseline-2018-09-02T022742.763006.json.bz2, baseline-2018-09-02T030122.647772.json.bz2, baseline-2018-

In [8]:
as_mappings = {}

data = urllib.request.urlopen("https://raw.githubusercontent.com/terose73/ICLab/main/as_mappings.txt?token=AIGKCL6YS7XXVB7SUXQI74C7W7MAU") 

for line in data: # files are iterable
    line = line.decode('utf-8').split(',')[0]
    as_num, name = line.split(' ', 1)
    as_mappings[as_num] = name.strip()

def find_mapping(x):
  res = as_mappings.get(x.split('.')[0])
  if not res:
    return np.nan
  return res

HTTPError: ignored

In [ ]:
# convert timestamps to pandas format in dataframe
df['server_t']= df['server_t'].apply(pd.Timestamp)

# get longer country names using pycountry API
df['country']= df['country'].apply(lambda x: pycountry.countries.get(alpha_2=x).name)

# get full as_name from number
df['as_name']= df['as_number'].apply(lambda x: find_mapping(str(x)))

In [ ]:
# drop unnecessary filename field
del df['filename']

In [ ]:
# replace {} and [] in http_reason with NaN
df['http_reason'] = df['http_reason'].mask(df['http_reason'].apply(str).eq('{}'))
df['http_reason'] = df['http_reason'].mask(df['http_reason'].apply(str).eq('[]'))

In [ ]:
# http status code of 1 doesn't make sense -> cast it to NaN

df['http_status'].mask(df['http_status'].eq(-1), inplace = True)

In [ ]:
# dns and dns_all of -2 doesn't make sense (and presents typing problems) -> cast it to NaN

def is_minus_2(x):
  return x != 'false' and x != 'true'

def convert(x):
  if x == "true" or x == "True":
    return True
  elif x == "false" or x == "False":
    return False
  return pd.NA

df['dns'] = df['dns'].mask( df['dns'].apply(lambda x: is_minus_2(x)) )
df['dns_all'] = df['dns_all'].mask(df['dns_all'].apply(lambda x: is_minus_2(x)) )

In [ ]:
# cast booleans to nullable type

df['dns_all'] = df['dns_all'].apply(convert).astype('boolean')
df['dns'] = df['dns'].apply(convert).astype('boolean')
df['packet_updated'] = df['packet_updated'].astype('boolean')
df['censored_updated'] = df['censored_updated'].astype('boolean')
df['block'] = df['block'].astype('boolean')

In [ ]:
# cast floats to nullable integers

df['as_number'] = df['as_number'].astype("Int64")
df['http_status'] = df['http_status'].astype("Int64")
df['body_len'] = df['body_len'].astype("Int64")

In [ ]:
# strip url http://, https://, and www

def strip_extra(x):
  return x.strip('http://').strip('https://').strip('/').strip('www.')

df['url'] = df['url'].apply(strip_extra)

In [ ]:
url_classification = open(dir + '/URL classification/type_domain_mapping.txt','r')

c = {}

for line in url_classification.readlines():

  classification, url = line.strip().split('|||')
  c[url.strip('www.')] = classification

print(c)
df['classification']= df['url'].apply(c.get)


In [ ]:
pd.set_option('display.max_colwidth', 0)
df.agg(['unique']).transpose()

In [ ]:
df.to_csv('ICLab_test1.csv', index=False)

In [ ]:
# Initialize Libra client
new_client = client('ICLab_test1.csv')

In [ ]:
new_client.neural_network_query('model', epochs=10)